### Pré-processamento do Dataset de radares:
- Remoção da coluna de velocidade média
- Formatação das colunas de tempo para 'datetime'
- Atribuição de tipo de variável 'float' para algumas colunas
- Cria coluna com final de placa para comparar com rodízio de veículos
- Anonimização dos veículos

In [ ]:
import numpy as np
import pandas as pd
import math as m
from datetime import datetime
import onetimepad

In [ ]:
# ===================== Constantes ======================

# Chave da criptografia 
# (Por motivos de segurança, o arquivo key.csv contém 
#  uma chave diferente da utilizada na anonimização dos dados reais)
key = list(pd.read_csv('key.csv'))[0]

In [ ]:
# ======================= Funcoes =======================

def cols2datetime(df):
    ''' Combina colunas de um dataframe contendo Ano, Mes, Dia, Hora e Minuto
        em uma variavel do tipo "datetime", salva em um objeto pd.Series'''
    datas = []
    for x in df.loc[:,'Ano':'Minuto'].values :
        datas.append(datetime(*x))
    return datas


def otp(sr):
    ''' Aplica uma criptografica de One-Time-Pad em um array do tipo pd.Series'''
    return sr.apply(onetimepad.encrypt, key=key)   

def sort_and_replace(sr):
    ''' Ordena um array de placas e substitui valores únicos por índices'''
    sr.sort_values(inplace=True)    
    sr = sr.reset_index()
    sr.loc[:,'Placa'] = sr.groupby('Placa').ngroup()+1
    sr.set_index('index', inplace=True)

    return sr
    
    
def encriptar_placas(sr):
    ''' Criptografia das placas:
        1 - Encripta as placas utilizando One-Time padding
        2 - Ordena os valores encriptados
        3 - Atribui um índice para cada grupo de valores únicos (groupby)
        4 - Substitui os valores encriptados pelos indices e deleta as placas encriptadas'''
    return sort_and_replace(otp(sr))
    
def str2num(df, COLUMN):
    
    for column in COLUMN:
        df.loc[:, column] = pd.to_numeric(df.loc[:, column], errors='coerce')
    
    return df
    
def processar_encriptar_um_dia(lote, mes, dia):

    #Carrega um arquivo CSV num DataFrame
    df_dia = pd.read_csv("Trafego_CSV/"+lote+"/"+mes+"c/"+dia+".csv")
    
    #Deleta a coluna 'Velocidade Media'
    df_dia = df_dia.iloc[:,:-1]    
    
    #Insere a coluna unificada de data e hora no DataFrame
    datahora = cols2datetime(df_dia)
    df_dia['Data e Hora'] = datahora
    df_dia = df_dia.drop(columns=['Ano', 'Mes', 'Dia', 'Hora', 'Minuto', 'Segundo'])
    
    #Altera as colunas numericas de str para float
    df_dia = str2num(df_dia, ['Especie', 'Classe', 'Comprimento', 'Velocidade', 'Ocupacao'])
    
    #Cria coluna com final das Placa
    df_dia['Final de Placa'] = df_dia['Placa'].str[6]
    
    #Encripta as Placas
    placa_bool = df_dia['Placa']!='       '
    df_dia.loc[placa_bool, 'Placa'] = encriptar_placas(df_dia.loc[placa_bool, 'Placa'])
    
    #Reodena Colunas
    df_dia = df_dia[['Lote', \
                     'Data e Hora',\
                     'Local',\
                     'Faixa',\
                     'Entrefaixa',\
                     'Registro',\
                     'Tipo de Registro',\
                     'Placa',\
                     'Final de Placa', \
                     'Especie',\
                     'Classe',\
                     'Comprimento',\
                     'Velocidade',\
                     'Ocupacao']]
    
    #Renomeia a Coluna da Placas para Veiculo
    df_dia = df_dia.rename(columns={'Placa':'Veiculo'})
    
    #Escreve um novo CSV
    df_dia.to_csv("Trafego_CSV/"+lote+"/"+mes+"g/"+dia+".csv", index=False, float_format='%.0f')
    
    return df_dia

In [ ]:
#----- Operacao -----

#parametros
lote = 'L3'
mes = '01'
'''
dias = ['01', '02', '03', '04', '05', \
        '06', '07', '08', '09', '10', \
        '11', '12', '13', '14', '15', \
        '16', '17', '18', '19', '20', \
        '21', '22', '23', '24', '25', \
        '26', '27', '28', '29', '30', '31']
'''
dias=['01']
for dia in dias:
    df = processar_encriptar_um_dia(lote, mes, dia)
    
#df.head(10)
#df.info()


In [ ]:
#----- Teste do Resultado ---
df_teste= pd.read_csv("Trafego_CSV/L3/01g/01.csv")
df_teste.head(10)